In [1]:
import torch
import pickle
import warnings
import numpy as np
import pandas as pd
import plotly.express as px
from copy import deepcopy
from tqdm import tqdm


from model import LR
from data import FairnessDataset, SyntheticDataset, GermanDataset
from ei_effort import Optimal_Effort, PGD_Effort
from ei_utils import *
from ei_model_test import EIModel, fair_batch_proxy, covariance_proxy

# warnings.filterwarnings('ignore')

In [2]:
# dataset = SyntheticDataset(seed=0)
dataset = GermanDataset(seed=0)
# dataset = Income

In [3]:
# torch.manual_seed(0)

seeds = list(range(3))
thetas = []
metrics = {
    'alpha': [],
    'Accuracy': [],
    'Fairness_Loss': [],
    'EI_Disparity': [],
    'alphas': []
}
for alpha in [0., 0.1, 0.5, 1.5, 2., 2.5, 5.0]:
    for seed in seeds:
        train_tensors, val_tensors, test_tensors = dataset.tensor(fold=seed, z_blind=False)
        train_dataset = FairnessDataset(*train_tensors, dataset.imp_feats)
        val_dataset = FairnessDataset(*val_tensors, dataset.imp_feats)
        test_dataset = FairnessDataset(*test_tensors, dataset.imp_feats)

        model = LR(train_dataset.X.shape[1])

        tau = 0.5
        delta = 0.5
        lamb = 0.8
        effort = PGD_Effort(delta, 50)
        proxy = covariance_proxy
        ei_model = EIModel(model, proxy, effort, tau)
        ei_model.train(train_dataset, lamb, alpha, abstol=1e-6)

        for module in ei_model.model.layers:
            if hasattr(module, 'weight'):
                weights_0 = module.weight.data
            if hasattr(module, 'bias'):
                bias_0 = module.bias.data

        theta_0 = torch.cat((weights_0[0], bias_0), 0)
        thetas.append(theta_0)

        pga_Y_hat, pga_Y_hat_max, pga_fair_loss = ei_model.predict(test_dataset, alpha, 1e-7)

        pga_accuracy, pga_ei_disparity = model_performance(
            test_dataset.Y.detach().numpy(), 
            test_dataset.Z.detach().numpy(), 
            pga_Y_hat, 
            pga_Y_hat_max, 
            tau
        )
        
        for module in ei_model.model_adv.layers:
            if hasattr(module, 'weight'):
                pga_weights_r = module.weight.data
            if hasattr(module, 'bias'):
                pga_bias_r = module.bias.data
        pga_theta_r = torch.cat((pga_weights_r[0], pga_bias_r), 0)

        Y_neg = pga_Y_hat < tau
        
        alphas = (pga_theta_r - theta_0).abs().detach().numpy()

        metrics['alpha'].append(alpha)
        metrics['Accuracy'].append(pga_accuracy)
        metrics['Fairness_Loss'].append(pga_fair_loss.detach().numpy())
        metrics['EI_Disparity'].append(pga_ei_disparity)
        metrics['alphas'].append(alphas)

Training [alpha=0.00; lambda=0.80; delta=0.50]: 100%|██████████| 100/100 [00:03<00:00, 31.48epochs/s]
Robust Training [alpha=0.10; lambda=0.80; delta=0.50]: 100%|██████████| 100/100 [00:24<00:00,  4.02epochs/s]
Robust Training [alpha=0.10; lambda=0.80; delta=0.50]: 100%|██████████| 100/100 [00:23<00:00,  4.18epochs/s]
Robust Training [alpha=0.10; lambda=0.80; delta=0.50]: 100%|██████████| 100/100 [00:24<00:00,  4.01epochs/s]
Robust Training [alpha=0.50; lambda=0.80; delta=0.50]: 100%|██████████| 100/100 [00:24<00:00,  4.16epochs/s]
Robust Training [alpha=0.50; lambda=0.80; delta=0.50]: 100%|██████████| 100/100 [00:25<00:00,  3.99epochs/s]
Robust Training [alpha=0.50; lambda=0.80; delta=0.50]: 100%|██████████| 100/100 [00:24<00:00,  4.14epochs/s]
Robust Training [alpha=1.50; lambda=0.80; delta=0.50]: 100%|██████████| 100/100 [00:23<00:00,  4.20epochs/s]
Robust Training [alpha=1.50; lambda=0.80; delta=0.50]: 100%|██████████| 100/100 [00:23<00:00,  4.21epochs/s]
Robust Training [alpha=1.5

In [4]:
df_metrics = pd.DataFrame(metrics)
df_alphas = df_metrics[['alpha', 'alphas']].copy()
df_alphas = df_alphas.sort_values(by='alpha').reset_index(drop=True)
df_metrics = df_metrics.drop(columns=['alphas'])

grouped_metrics = df_metrics.groupby('alpha').agg(['mean', 'std'])
grouped_metrics

Accuracy           Fairness_Loss           EI_Disparity          
           mean       std          mean       std         mean       std
alpha                                                                   
0.0    0.745000  0.013229      0.000076  0.000038     0.043921  0.024148
0.1    0.750000  0.018028      0.002104  0.000518     0.261610  0.067225
0.5    0.735000  0.035000      0.016798  0.005684     0.460885  0.082300
1.5    0.750000  0.013229      0.032457  0.005587     0.504703  0.086135
2.0    0.750000  0.005000      0.036190  0.008752     0.463468  0.039930
2.5    0.750000  0.025981      0.039831  0.003455     0.524139  0.080341
5.0    0.753333  0.016073      0.050229  0.005314     0.552124  0.054264

In [6]:
df_metrics = pd.DataFrame(metrics)
df_alphas = df_metrics[['alpha', 'alphas']].copy()
df_alphas = df_alphas.sort_values(by='alpha').reset_index(drop=True)
df_metrics = df_metrics.drop(columns=['alphas'])

grouped_metrics = df_metrics.groupby('alpha').agg(['mean', 'std'])
grouped_metrics

Accuracy           Fairness_Loss           EI_Disparity          
           mean       std          mean       std         mean       std
alpha                                                                   
0.0    0.745000  0.013229      0.000076  0.000038     0.043921  0.024148
0.1    0.750000  0.018028      0.002104  0.000518     0.261610  0.067225
0.5    0.735000  0.035000      0.016797  0.005684     0.460885  0.082300
1.5    0.750000  0.013229      0.032456  0.005587     0.504703  0.086135
2.0    0.750000  0.005000      0.036190  0.008752     0.463468  0.039930
2.5    0.750000  0.025981      0.039831  0.003455     0.524139  0.080341
5.0    0.753333  0.016073      0.050228  0.005314     0.552124  0.054264

In [10]:
df_metrics = pd.DataFrame(metrics)
thetas

[tensor([ 0.4030, -0.3843,  0.1069, -0.0501, -0.0497,  0.2472,  0.0896, -0.0266,
          0.0257,  0.2969, -0.0381, -0.2188, -0.0528,  0.0013, -0.0210,  0.0488,
          0.0350,  0.3128,  0.3879,  0.0078,  0.0872]),
 tensor([ 0.5505, -0.1659,  0.2708, -0.0034, -0.2420,  0.2106,  0.0321, -0.1935,
          0.0383,  0.2092,  0.0494, -0.1972,  0.0314,  0.0038, -0.2194, -0.0919,
          0.1434,  0.2745,  0.6159,  0.0957, -0.1387]),
 tensor([ 0.4485, -0.4175,  0.0809,  0.0169, -0.1348,  0.2895,  0.0501, -0.1330,
          0.1136,  0.3839, -0.0087, -0.1587,  0.2289, -0.0703,  0.0183, -0.1003,
         -0.1794,  0.1165,  0.4847,  0.2631, -0.1477]),
 tensor([ 0.5424, -0.3246,  0.4290,  0.0028, -0.2765,  0.2669,  0.2139, -0.4896,
          0.1085,  0.5045,  0.0183, -0.2598,  0.1820,  0.4176, -0.1679,  0.1443,
         -0.4314,  0.3810,  0.7839, -0.0747, -0.4061]),
 tensor([ 0.4695, -0.2477,  0.1429, -0.0165, -0.1579,  0.2040,  0.1513, -0.0665,
          0.1179,  0.1690, -0.1487, -0.1804,  0

In [7]:
for alpha in [0., 0.1, 0.5, 1.5, 2., 2.5, 5.0]:
    pga_Y_hat, pga_Y_hat_max, pga_fair_loss = ei_model.predict(test_dataset, alpha, 1e-7)
    pga_accuracy, pga_ei_disparity = model_performance(test_dataset.Y.detach().numpy(), test_dataset.Z.detach().numpy(), pga_Y_hat, pga_Y_hat_max, tau)
    for module in ei_model.model_adv.layers:
        if hasattr(module, 'weight'):
            pga_weights_r = module.weight.data
        if hasattr(module, 'bias'):
            pga_bias_r = module.bias.data
    pga_theta_r = torch.cat((pga_weights_r[0], pga_bias_r), 0)

    Y_neg = pga_Y_hat<tau
    
    alphas = (pga_theta_r-theta_0).abs()
    print(f'[{dataset.__class__.__name__}] PGA Results {"(Covariance)" if proxy == covariance_proxy else "(Loss-based)"}')
    print(f'alpha               |   {alpha}')
    print(f'Accuracy            |   {pga_accuracy:.5f}')
    print(f'Fairness Loss       |   {pga_fair_loss:.5f}')
    print(f'EI Disparity        |   {pga_ei_disparity:.5f}')
    # print(f'theta_0             |   {np.round(theta_0, 4)}')
    # print(f'theta_r             |   {np.round(pga_theta_r, 4)}')
    print(f'alphas              |   {np.round(alphas, 4)}')
    print('-'*32)
    print()

[GermanDataset] PGA Results (Covariance)
alpha               |   0.0
Accuracy            |   0.76000
Fairness Loss       |   0.00007
EI Disparity        |   0.08213
alphas              |   tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
--------------------------------

[GermanDataset] PGA Results (Covariance)
alpha               |   0.1
Accuracy            |   0.76000
Fairness Loss       |   0.00214
EI Disparity        |   0.23188
alphas              |   tensor([0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
        0.1000, 0.0465, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
        0.0000, 0.1000, 0.1000])
--------------------------------

[GermanDataset] PGA Results (Covariance)
alpha               |   0.5
Accuracy            |   0.76000
Fairness Loss       |   0.01641
EI Disparity        |   0.49758
alphas              |   tensor([0.1300, 0.5000, 0.5000, 0.1430, 0.5000, 0.5000, 0.5000, 0.5000, 0.4951,

In [8]:
for alpha in [0., 0.1, 0.5, 1.5, 2., 2.5, 5.0]:
    pga_Y_hat, pga_Y_hat_max, pga_fair_loss = ei_model.predict(test_dataset, alpha, 1e-7)
    pga_accuracy, pga_ei_disparity = model_performance(test_dataset.Y.detach().numpy(), test_dataset.Z.detach().numpy(), pga_Y_hat, pga_Y_hat_max, tau)
    for module in ei_model.model_adv.layers:
        if hasattr(module, 'weight'):
            pga_weights_r = module.weight.data
        if hasattr(module, 'bias'):
            pga_bias_r = module.bias.data
    pga_theta_r = torch.cat((pga_weights_r[0], pga_bias_r), 0)

    Y_neg = pga_Y_hat<tau
    
    alphas = (pga_theta_r-theta_0).abs()
    print(f'[{dataset.__class__.__name__}] PGA Results {"(Covariance)" if proxy == covariance_proxy else "(Loss-based)"}')
    print(f'alpha               |   {alpha}')
    print(f'Accuracy            |   {pga_accuracy:.5f}')
    print(f'Fairness Loss       |   {pga_fair_loss:.5f}')
    print(f'EI Disparity        |   {pga_ei_disparity:.5f}')
    # print(f'theta_0             |   {np.round(theta_0, 4)}')
    # print(f'theta_r             |   {np.round(pga_theta_r, 4)}')
    print(f'alphas              |   {np.round(alphas, 4)}')
    print('-'*32)
    print()

[GermanDataset] PGA Results (Covariance)
alpha               |   0.0
Accuracy            |   0.76000
Fairness Loss       |   0.00007
EI Disparity        |   0.08213
alphas              |   tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
--------------------------------

[GermanDataset] PGA Results (Covariance)
alpha               |   0.1
Accuracy            |   0.76000
Fairness Loss       |   0.00214
EI Disparity        |   0.23188
alphas              |   tensor([0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
        0.1000, 0.0465, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
        0.0000, 0.1000, 0.1000])
--------------------------------

[GermanDataset] PGA Results (Covariance)
alpha               |   0.5
Accuracy            |   0.76000
Fairness Loss       |   0.01641
EI Disparity        |   0.49758
alphas              |   tensor([0.1300, 0.5000, 0.5000, 0.1430, 0.5000, 0.5000, 0.5000, 0.5000, 0.4951,